# Introduction to Google OR-Tools

Google OR-Tools is a powerful suite designed to tackle combinatorial optimization problems, ranging from routing and scheduling to supply chain and logistics planning. At its core, OR-Tools provides a rich set of solvers for Linear Programming (LP), Mixed Integer Programming (MIP), Constraint Programming (CP), and more, making it versatile for both industry practitioners and researchers.

This toolkit is designed with efficiency and scalability in mind, offering APIs in several programming languages, including Python, C++, and Java, to make its powerful optimization engines accessible to a wide audience. Whether you're optimizing delivery routes for a fleet of vehicles or scheduling shifts for a nursing staff, OR-Tools can help you find the best solution given a set of constraints and objectives.

OR-Tools stands out for its ability to solve large-scale optimization problems quickly and with high precision. It employs advanced algorithms and techniques to navigate the complex landscape of feasible solutions, efficiently finding the optimal or near-optimal solution.

By leveraging OR-Tools, developers and decision-makers can implement optimization logic into their applications, enabling them to make more informed decisions, reduce costs, and improve operational efficiency. As we explore OR-Tools further, we'll delve into its capabilities, understand how to model different types of optimization problems, and learn how to harness its power to solve real-world challenges.

## Nurse Scheduling with Constraint Programming

In the field of healthcare management, the task of creating schedules for nursing staff that are both effective and considerate of their needs is crucial yet challenging. It involves allocating shifts in a manner that guarantees each shift is adequately staffed while also accommodating various constraints such as maximum working hours, personal preferences, and required rest periods. This issue requires a solution that not only meets the operational demands but also values the nursing staff's well-being.

We apply Constraint Programming (CP), an approach in operations research designed for tackling problems with multiple constraints. CP enables us to frame our scheduling issue with variables (shifts assigned to nurses), constraints (like maximum shifts per nurse and staffing needs), and an objective (to minimize unfulfilled preferences or the use of slack).

The model we develop starts with basic assignments of nurses to shifts, ensuring no individual is overloaded. We then incorporate additional layers of complexity, like ensuring a minimum number of staff for each shift, using slack variables to enhance the model's ability to find workable solutions under stringent conditions.

This section aims to provide a foundation in how to approach and solve the nurse scheduling problem through Constraint Programming. We delve into the essential techniques and strategies that allow us to create a schedule that efficiently meets hospital needs while also considering the preferences and welfare of the nursing staff.



### Scenario Overview: Nurse Scheduling Optimization

In our analysis, we address the widespread and intricate issue of scheduling nursing staff to achieve continuous hospital operation. We propose a model for a small hospital that needs to create an effective weekly schedule for its nurses, considering various operational and personnel factors.

The hospital operates across three shifts each day: Morning (7 AM to 3 PM), Evening (3 PM to 11 PM), and Night (11 PM to 7 AM). The objective is to assign nurses to these shifts in a way that fulfills the hospital's variable staffing requirements for each shift and day, while adhering to important constraints:

Nurses are limited to working up to 5 shifts a week to ensure they receive sufficient rest and maintain a work-life balance.
Preferences and availability of each nurse are taken into account to improve job satisfaction.
Nurses are not scheduled for more than three consecutive shifts to prevent burnout and maintain a high level of patient care.
The goal is to craft a schedule using Constraint Programming that meets these operational and human-centric criteria, effectively navigating through the complexities of shift scheduling.

### Nurse Scheduling Problem: Mathematical Model

#### Sets and Indices
- $N$: Set of nurses, indexed by $n$.
- $D$: Set of days in the scheduling period, indexed by $d$.
- $S$: Set of shifts within a day, indexed by $s$.

#### Parameters
- $M$: Maximum number of shifts a nurse can work over the scheduling period.
- $R_{d,s}$: Required number of nurses for shift $s$ on day $d$.
- $P_{n,s}$: Preference score of nurse $n$ for working shift $s$, where a higher score indicates a stronger preference.

#### Decision Variables
- $x_{n,d,s}$: Binary variable that equals 1 if nurse $n$ is assigned to shift $s$ on day $d$, and 0 otherwise.

#### Objective
Minimize the total negative preference across all nurse-shift assignments, ensuring nurses are assigned to shifts they prefer less as infrequently as possible.
$$\min \sum_{n \in N} \sum_{d \in D} \sum_{s \in S} (1 - P_{n,s}) \cdot x_{n,d,s}$$

#### Constraints

1. **Shift Requirement**: Each shift on each day must be filled with the required number of nurses.
$$\sum_{n \in N} x_{n,d,s} \geq R_{d,s}, \quad \forall d \in D, \forall s \in S$$

2. **Maximum Shifts**: No nurse works more than the maximum number of shifts.
$$\sum_{d \in D} \sum_{s \in S} x_{n,d,s} \leq M, \quad \forall n \in N$$

3. **One Shift Per Day**: Each nurse can work at most one shift per day.
$$\sum_{s \in S} x_{n,d,s} \leq 1, \quad \forall n \in N, \forall d \in D$$

4. **Preference Score**: Only consider assignments that match nurse preferences, optional constraint for enhancing model flexibility.
$$P_{n,s} \cdot x_{n,d,s} \geq \text{Preference Threshold}, \quad \text{optional, for selected } n, d, s$$

This mathematical model aims to provide an optimal scheduling solution that meets operational requirements while considering nurse preferences and constraints.


In [1]:
from ortools.sat.python import cp_model

def model_with_constraint_relaxation():
    # Model
    model = cp_model.CpModel()

    # Parameters
    num_nurses = 10
    num_shifts = 3  # Morning, Evening, Night
    num_days = 7
    shifts_per_nurse = 5  # Total shifts each nurse must be assigned

    shift_requirements = {0: 4, 1: 2, 2: 2}  # Morning, Evening, Night requirements

    # Variables
    shifts = {(nurse, day, shift): model.NewBoolVar(f'nurse{nurse}_day{day}_shift{shift}')
              for nurse in range(num_nurses)
              for day in range(num_days)
              for shift in range(num_shifts)}
    
    slack_vars = {(day, shift): model.NewIntVar(0, num_nurses, f'slack_day{day}_shift{shift}')
                  for day in range(num_days) for shift in range(num_shifts)}

    # Constraints
    # Each nurse works exactly 'shifts_per_nurse' shifts in a week
    for nurse in range(num_nurses):
        model.Add(sum(shifts[nurse, day, shift] for day in range(num_days) for shift in range(num_shifts)) == shifts_per_nurse)

    # Each shift on each day has at least the minimum required number of nurses (with slack)
    for day in range(num_days):
        for shift in range(num_shifts):
            model.Add(sum(shifts[nurse, day, shift] for nurse in range(num_nurses)) + slack_vars[day, shift] >= shift_requirements[shift])

    # Objective: Minimize the total slack used
    model.Minimize(sum(slack_vars[day, shift] for day in range(num_days) for shift in range(num_shifts)))

    # Solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("Solution with constraint relaxation found.")
        for day in range(num_days):
            for shift in range(num_shifts):
                print(f'Day {day}, Shift {shift}: ', end='')
                for nurse in range(num_nurses):
                    if solver.Value(shifts[nurse, day, shift]):
                        print(f'Nurse {nurse} ', end='')
                slack = solver.Value(slack_vars[day, shift])
                if slack > 0:
                    print(f"(Slack used: {slack})", end='')
                print()
    else:
        print("No solution found with constraint relaxation.")

model_with_constraint_relaxation()


Solution with constraint relaxation found.
Day 0, Shift 0: Nurse 4 Nurse 6 Nurse 7 Nurse 8 
Day 0, Shift 1: Nurse 4 Nurse 7 
Day 0, Shift 2: Nurse 3 Nurse 6 
Day 1, Shift 0: Nurse 3 Nurse 6 Nurse 7 Nurse 8 
Day 1, Shift 1: Nurse 6 Nurse 8 
Day 1, Shift 2: Nurse 2 Nurse 9 
Day 2, Shift 0: Nurse 0 Nurse 1 (Slack used: 2)
Day 2, Shift 1: Nurse 4 Nurse 5 
Day 2, Shift 2: Nurse 3 Nurse 8 
Day 3, Shift 0: Nurse 0 Nurse 1 Nurse 9 (Slack used: 1)
Day 3, Shift 1: Nurse 3 Nurse 7 
Day 3, Shift 2: Nurse 6 Nurse 8 
Day 4, Shift 0: Nurse 0 Nurse 1 Nurse 9 (Slack used: 1)
Day 4, Shift 1: Nurse 3 Nurse 5 
Day 4, Shift 2: Nurse 7 Nurse 9 
Day 5, Shift 0: Nurse 0 Nurse 1 Nurse 9 (Slack used: 1)
Day 5, Shift 1: Nurse 2 Nurse 5 
Day 5, Shift 2: Nurse 2 Nurse 5 
Day 6, Shift 0: Nurse 0 Nurse 1 Nurse 4 (Slack used: 1)
Day 6, Shift 1: Nurse 2 Nurse 5 
Day 6, Shift 2: Nurse 2 Nurse 4 
